# Mask R-CNN - Inspect Training Data

Inspect and visualize data loading and pre-processing code.

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import skimage.io

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Configurations

Run one of the code blocks below to import and load the configurations to use.

In [2]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = "../../../COCO2017"  # TODO: enter value here

## Dataset

In [3]:
# Load dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=16.25s)
creating index...
index created!
Image Count: 117266
Class Count: 81
  0. BG                                                
  1. person                                            
  2. bicycle                                           
  3. car                                               
  4. motorcycle                                        
  5. airplane                                          
  6. bus                                               
  7. train                                             
  8. truck                                             
  9. boat                                              
 10. traffic light                                     
 11. fire hydrant                                      
 12. stop sign                                         
 13. parking meter                                     
 14. bench                                             
 15. bird                               

## Cut images with GT bbs

Load images and cut. Save the images and masks.

In [4]:
# Load and display random samples
for image_id in dataset.image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    for i, c in enumerate(class_ids):
        if c == 1:
            file_to_save = str(image_id) + '_' + str(i)+'.jpg'
            file_npy = str(image_id) + '_' + str(i)+'.npy'
            gt_bbox = utils.extract_bboxes(mask)
            bbox = gt_bbox[i,:]

            y1, x1, y2, x2 = bbox
            if y2-y1 > 2*(x2-x1):
                x2 = x1 + int((y2-y1)/2)
            else:
                y2 = y1 + 2*(x2-x1)    
            if y2 > image.shape[0] or x2 > image.shape[1]:
                continue
            if y2-y1 < 256 or x2-x1 < 128:
                continue
            
            dst=image[y1:y2,x1:x2,:]
            dst = skimage.transform.resize(dst,(256,128))

            mask_save = mask[:,:,i]*1.0
            mask_save = mask_save[y1:y2,x1:x2]
            mask_save = skimage.transform.resize(mask_save,(256,128))
            
            skimage.io.imsave(os.path.join('./imgs256/',file_to_save),dst)
            np.save(os.path.join('./imgs256/',file_npy), mask_save)
            
    if image_id%1000==0:
        print(image_id/dataset.image_ids[-1])

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


0.0


KeyboardInterrupt: 